<a href="https://colab.research.google.com/github/MarianaDuartee/ProjetoFinal/blob/main/3_SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Instalando  dependencias

In [ ]:
!pip install pyspark
!pip install gcsfs
!pip install pandas


##Importando dataframes, bibliotecas e abrindo Sparksession

In [40]:
from pyspark.sql import SparkSession
from pyspark import SQLContext
from google.cloud import storage
from pyspark.sql.types import *
from pyspark.sql.types import DateType

import pyspark
import pyspark.sql.functions as F
import os
import gcsfs
import pandas as pd

##Criando uma conexão Spark e configurando chave de serviço



In [2]:
spark = SparkSession.builder\
.master('local')\
.appName('Projeto_Final')\
.config('spark.ui.enable', 'true')\
.config('spark.ui.port', '4050')\
.getOrCreate()

spark

serviceaccount = '/content/soulcode-projeto-final-4b88bea6e07a.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = serviceaccount

## **EXTRAINDO INFORMAÇÕES E ANALISANDO DATAFRAMES**

Separar as planilhas e alterar o arquivo da planilha 6. Acrescentar planilha 7

df_ 1 - Planilha de despesas com segurança publica

In [9]:
file_path_1 = 'gs://data_lake_ingest_data/2_temp/temp_pandas_despesas_normalizado.csv'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_1):
    data = pd.read_csv(file_path_1, sep=',', encoding='UTF-8', header=0)

df_1 = spark.createDataFrame(data)
df = df_1.createOrReplaceTempView('df_1')

#df_1.printSchema()
#df_1.dtypes

df_1.show(5, truncate=False)


+--------+------------------+------------+------------------+------------+------------------+---------+------------------+
|UF      |Despesas2016      |Despesas2017|Despesas2018      |Despesas2019|Despesas2020      |Variacao%|Previsao2021|Media|
+--------+------------------+------------+------------------+------------+------------------+---------+------------------+
|Acre    |498.535           |568.361     |627.351           |692.57      |493.75300000000004|8.5      |576.114           |
|Amapá   |475.32300000000004|500.227     |489.228           |606.366     |752.252           |52.3     |564.6792          |
|Amazonas|1604.7320000000002|1578.957    |1862.6689999999999|2108.46     |2101.447          |60.4     |1851.253          |
|Pará    |2551.741          |2579.916    |2903.2            |2943.448    |2966.553          |58.0     |2788.9716         |
|Rondônia|874.8739999999999 |895.868     |903.2539999999999 |853.971     |999.044           |-14.7    |905.4022          |
+--------+------

df_2 - Numero de ocorrencias 

In [10]:
file_path_2 = 'gs://data_lake_ingest_data/2_temp/temp_pandas_ocorrencias_normalizado.json'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_2):
    data = pd.read_json(file_path_2, encoding='UTF-8')

df_2 = spark.createDataFrame(data)
df = df_2.createOrReplaceTempView('df_2')

#df_2.printSchema()
#df_2.dtypes

df_2.show(5, truncate=False)


+----+-------------------------------+----+-------+-----------+
|UF  |TipoCrime                      |Ano |Mes    |Ocorrencias|
+----+-------------------------------+----+-------+-----------+
|Acre|Estupro                        |2021|janeiro|39         |
|Acre|Furto de veículo               |2021|janeiro|55         |
|Acre|Homicídio doloso               |2021|janeiro|14         |
|Acre|Lesão corporal seguida de morte|2021|janeiro|0          |
|Acre|Roubo a instituição financeira |2021|janeiro|0          |
+----+-------------------------------+----+-------+-----------+
only showing top 5 rows



df_3 - Tipo de Ocorrencias com sexo e numero de vitimas Brasil 2021

In [11]:
file_path_3 = 'gs://data_lake_ingest_data/2_temp/temp_pandas_vitimas_normalizado.json'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_3):
    data = pd.read_json(file_path_3, encoding='UTF-8')

df_3 = spark.createDataFrame(data)
df = df_3.createOrReplaceTempView('df_3')

#df_3.printSchema()
#df_3.dtypes

df_3.show(10, truncate=False)


+----+----------------+----+---------+----------+-------+
|UF  |TipoCrime       |Ano |Mes      |SexoVitima|Vitimas|
+----+----------------+----+---------+----------+-------+
|Acre|Homicídio doloso|2021|janeiro  |Feminino  |1      |
|Acre|Homicídio doloso|2021|janeiro  |Masculino |13     |
|Acre|Homicídio doloso|2021|janeiro  |Sexo NI   |0      |
|Acre|Homicídio doloso|2021|fevereiro|Feminino  |4      |
|Acre|Homicídio doloso|2021|fevereiro|Masculino |12     |
|Acre|Homicídio doloso|2021|fevereiro|Sexo NI   |0      |
|Acre|Homicídio doloso|2021|março    |Feminino  |2      |
|Acre|Homicídio doloso|2021|março    |Masculino |24     |
|Acre|Homicídio doloso|2021|março    |Sexo NI   |0      |
|Acre|Homicídio doloso|2021|abril    |Feminino  |2      |
+----+----------------+----+---------+----------+-------+
only showing top 10 rows



In [12]:
file_path_4 = 'gs://data_lake_ingest_data/2_temp/temp_pandas_ocorrencias_normalizado.json'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_4):
    data = pd.read_json(file_path_4, encoding='UTF-8')

df_4 = spark.createDataFrame(data)
df = df_4.createOrReplaceTempView('df_4')

#df_4.printSchema()
#df_4.dtypes

df_4.show(5, truncate=False)


+----+-------------------------------+----+-------+-----------+
|UF  |TipoCrime                      |Ano |Mes    |Ocorrencias|
+----+-------------------------------+----+-------+-----------+
|Acre|Estupro                        |2021|janeiro|39         |
|Acre|Furto de veículo               |2021|janeiro|55         |
|Acre|Homicídio doloso               |2021|janeiro|14         |
|Acre|Lesão corporal seguida de morte|2021|janeiro|0          |
|Acre|Roubo a instituição financeira |2021|janeiro|0          |
+----+-------------------------------+----+-------+-----------+
only showing top 5 rows



df_5 - Ocorrencias vitimas por estado e tipo de ocorrencia

In [13]:
#ARQUIVO 5
file_path_5 = 'gs://data_lake_ingest_data/2_temp/temp_pandas_ocorrencia_vitimas_porAnoEstado.csv'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_5):
    data = pd.read_csv(file_path_5, sep=',', encoding='UTF-8')

df_5 = spark.createDataFrame(data)
df = df_5.createOrReplaceTempView('df_5')

#df_5.printSchema()
#df_5.dtypes

df_5.show(5, truncate=False)


+----+----+-----------------------------------+-----------+-------+
|UF  |Ano |TipoCrime                          |Ocorrencias|Vitimas|
+----+----+-----------------------------------+-----------+-------+
|Acre|2016|Homicídio doloso                   |425        |354    |
|Acre|2017|Homicídio doloso                   |221        |232    |
|Acre|2017|Lesão corporal seguida de morte    |0          |3      |
|Acre|2017|Roubo seguido de morte (latrocínio)|10         |15     |
|Acre|2018|Homicídio doloso                   |396        |394    |
+----+----+-----------------------------------+-----------+-------+
only showing top 5 rows



df-6 - Frequencia escolar Brasil anos de 2016 a 2019 por faixa de idade

In [14]:
file_path_6 = 'gs://data_lake_ingest_data/1_input/Tabela_frequencia_escolar.xlsx'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_6):
    data = pd.read_excel(file_path_6, header=0)

df_6 = spark.createDataFrame(data)
df = df_6.createOrReplaceTempView('df_6')

#df_6.printSchema()
#df_6.dtypes

df_6.show(5, truncate=False)


+----------------------------------------------------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+----------------------+----------------------+----------------------+----------------------+---------------------+
|Grandes Regiões, Unidades da Federação e Municípios das Capitais|15 a 17 anos - 2019|15 a 17 anos - 2018|15 a 17 anos - 2017|15 a 17 anos - 2016|Media 15 a 17 anos|18 a 24 anos - 2019|18 a 24 anos - 2018|18 a 24 anos - 2017|18 a 24 anos - 2016|média 18 a 24 anos|25 anos ou mais - 2019|25 anos ou mais - 2018|25 anos ou mais - 2017|25 anos ou mais - 2016|média 25 anos ou mais|
+----------------------------------------------------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-----------------

##Realizando algumas analises

Total de ocorrencias criminais por Estado (nos anos de 2016 a 2021) Está correto 2021??? De quando foi essa analise?

In [15]:
spark.sql(
'''
SELECT UF, SUM(Ocorrencias) AS Total_Ocorrencias_por_estado
FROM df_5
GROUP BY UF
ORDER BY UF ASC
'''
).show()


+-------------------+----------------------------+
|                 UF|Total_Ocorrencias_por_estado|
+-------------------+----------------------------+
|               Acre|                        1776|
|            Alagoas|                        7642|
|              Amapá|                        1599|
|           Amazonas|                        6098|
|              Bahia|                       31016|
|              Ceará|                       19895|
|   Distrito Federal|                        2547|
|     Espírito Santo|                        6549|
|              Goiás|                       11116|
|           Maranhão|                       10276|
|        Mato Grosso|                        5299|
| Mato Grosso do Sul|                        2762|
|       Minas Gerais|                       17835|
|             Paraná|                       10102|
|            Paraíba|                        6411|
|               Pará|                       17664|
|         Pernambuco|          

Quantidade de vitimas do sexo feminino por tipo de crime

In [16]:
spark.sql(
  
'''
SELECT TipoCrime,
SUM(Vitimas) AS Feminino
FROM df_3
WHERE SexoVitima = 'Feminino'
GROUP BY TipoCrime
'''

).show()

+--------------------+--------+
|           TipoCrime|Feminino|
+--------------------+--------+
|Roubo seguido de ...|    1056|
|    Homicídio doloso|   20834|
|Lesão corporal se...|     527|
+--------------------+--------+



Quantidade de vitimas do sexo maculino por tipo de crime

In [41]:
spark.sql(
'''
SELECT TipoCrime,
CASE WHEN SexoVitimas = 'Feminino', THEN SUM (Vitimas) AS Feminino;
CASE WHEN SexoVitimas = 'Masculino', THEN SUM (Vitimas) AS Masculino;
ELSE SUM (Vitimas) AS NI
FROM df_3
GROUP BY TipoCrime
'''
).show()

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



ParseException: ignored